# Using Python and Selenium to get coordinates from street addresses :

![Melbourne](https://cdn-images-1.medium.com/max/1200/1*oNUuJJUC_J7hCzbZ0zds2Q.jpeg)

This is a step by step article on how to use **Python** and **Selenium** to scrape **coordinates  data** (Latitude and Longitude values) from Google Maps **based on street addresses**. In this case example, I'm going to work with an official dataset containing the street addresses of all **Australian charities and nonprofits**. At the end, I'll proceed to map all the charities and nonprofits in the city of Melbourne using **Folium**, to demonstrate what can be done with the newly acquired coordinates data.

---

# The packages and the setup :

We'll need the [Selenium](https://pypi.org/project/selenium/) python package to perform the data scraping. If you don't have it yet, you can install it [using pip](https://www.youtube.com/watch?v=FKwicZF7xNE) : `pip install selenium` . 

We'll also need a WebDriver in order to interact with the browser, so you have to [go here and download it](https://chromedriver.chromium.org/downloads) to your machine (make sure it's compatible with your current Chrome version).
Now let's do the import :

In [1]:
from selenium import webdriver

Let's also get [tqdm](https://github.com/tqdm/tqdm), an essential progressbar python package. It is very useful to estimate how much time the web scraping part of your code will take (i'm using tqdm_notebook because I'm working in a Jupyter Notebook) :

In [2]:
from tqdm import tqdm_notebook as tqdmn

In addition to that, we'll need [Pandas](https://pandas.pydata.org/) to read and manipulate the dataset :

In [3]:
import pandas as pd

Finally, we'll get [Folium](https://pypi.org/project/folium/) to map coordinates data on a map (it can be installed using `pip install folium`):

In [4]:
import folium

This is a CSS code to format our dataframes for added contrast between the indexes and the data itself (more info on that [here](https://medium.com/analytics-vidhya/5-css-snippets-for-better-looking-pandas-dataframes-ad808e407894?source=friends_link&sk=32c05ad43eb79378f6b4992ae809953a)) :

In [5]:
%%HTML
<style>.dataframe th{background: rgb(63,87,124);background: linear-gradient(180deg, rgba(63,87,124,1) 0%, rgba(101,124,161,1) 100%, rgba(0,212,255,1) 100%);;
padding: 10px;font-family: monospace;font-size: 110%;color: white;border:1px dashed white;text-align:left !important;
-moz-border-radius: 3x;-webkit-border-radius: 3px;}.dataframe thead{border:none; !important;}</style>

---

# The dataset: ACNC charities and nonprofits registry :

Australia is ranked 4th worldwide by the CAF World Giving Index in its [10th edition](https://www.cafonline.org/about-us/publications/2019-publications/caf-world-giving-index-10th-edition) that covers the last 10 years. By visiting the [ACNC's website](https://www.acnc.gov.au/), you'll quickly notice how easy they make it for researchers to access all the data about charities and nonprofits in Australia.

The dataset we'll be using can be downloaded here (along with a useful user notes document explaining the variables ):

* [ACNC Charity Register dataset](http://data.gov.au/dataset/acnc-register): Excel file XLSX | 7.7 Mb | 74155 rows and 60 columns as of 23/01/2020

The dataset presents many interesting aspects about the charities and nonprofits in Australia: unique identifier, legal name, address (for correspondence), registration date, size, purpose, beneficiaries and more (read the user notes for more info).

Here, we're mainly interested in the **street address features** which span several columns: `Address_Line_1`,  `Address_Line_2`,  `Address_Line_3`,  `Town_City`,  `State`,  `Postcode` and `Country`. Also, for simplicity, we will only look into charities and nonprofits in the city of Melbourne.

---

# Data cleaning and preparation :

First, let's read in the dataset we previously downloaded from the ACNC website using Pandas :

In [6]:
# we set this Pandas option to make all 60 columns visible to us :
pd.set_option('display.max_columns', 60)

acnc = pd.read_excel('data/datadotgov_main.xlsx', keep_default_na=False)

Make sure to supply the path where the Excel file resides on your machine. The `keep_default_na` parameter is set to `False` so we'll get empty values instead of NaN when certain values are missing. This will be useful later when we'll combine all the street address variables into one variable.

In [7]:
acnc.head()

,ABN,Charity_Legal_Name,Other_Organisation_Names,Operating_Countries,Address_Type,Address_Line_1,Address_Line_2,Address_Line_3,Town_City,State,Postcode,Country,Charity_Website,Registration_Date,Date_Organisation_Established,Charity_Size,Number_of_Responsible_Persons,Financial_Year_End,Operates_in_ACT,Operates_in_NSW,Operates_in_NT,Operates_in_QLD,Operates_in_SA,Operates_in_TAS,Operates_in_VIC,Operates_in_WA,PBI,HPC,Preventing_or_relieving_suffering_of_animals,Advancing_Culture,Advancing_Education,Advancing_Health,Promote_or_oppose_a_change_to_law__government_poll_or_prac,Advancing_natual_environment,Promoting_or_protecting_human_rights,Purposes_beneficial_to_ther_general_public_and_other_analogous,Promoting_reconciliation__mutual_respect_and_tolerance,Advancing_Religion,Advancing_social_or_public_welfare,Advancing_security_or_safety_of_Australia_or_Australian_public,Another_purpose_beneficial_to_the_community,Aboriginal_or_TSI,Aged_Persons,Children,Communities_Overseas,Ethnic_Groups,Gay_Lesbian_Bisexual,General_Community_in_Australia,Men,Migrants_Refugees_or_Asylum_Seekers,Pre_Post_Release_Offenders,People_with_Chronic_Illness,People_with_Disabilities,People_at_risk_of_homelessness,Unemployed_Person,Veterans_or_their_families,Victims_of_crime,Victims_of_Disasters,Women,Youth
0,11000047950,Sydney Missionary & Bible College,,,Business,43 Badminton Rd,,,Croydon,New South Wales,2132,Australia,,03/12/2012,,Large,9,31-Dec,,Y,,,,,,,,,,,Y,,,,,,,Y,,,,,,,Y,Y,,Y,,,,,,,,,,,,
1,11000073870,Integricare,,,Business,,,,,,,Australia,http://www.integricare.org.au,03/12/2012,31/12/1881,Large,8,30-Jun,,Y,,,,,,,,,,,,,,,,Y,,,Y,,Y,,,,,,,,,,,,,,,,,,,
2,11000761571,Australian Missionary Tidings,,"BRA, COD, COL, ESP, FJI, IND, ITA, KHM, MEX, N...",Business,Po Box 565,,,Mount Gravatt,Queensland,4122,Australia,,03/12/2012,31/12/1909,Large,12,31-Dec,,,,Y,,,,,,,,,,,,,,,,Y,,,,,,,Y,Y,,,,,,,,,,,,Y,,
3,11001233790,"The Greek Orthodox Church & Community Of The ""...",,,Business,11 Steel Street,,,HAMILTON,NSW,2303,Australia,,03/12/2012,,Medium,5,30-Jun,,Y,,,,,,,,,,,,,,,,,,Y,,,,Y,,,Y,Y,Y,Y,Y,Y,,,Y,,,,,Y,Y,Y
4,11001241005,Wangarang Industries Limited,,,Business,PO Box 8133,,,Orange,NSW,2800,Australia,,03/12/2012,01/01/1975,Large,8,30-Jun,,Y,,,,,,,Y,,,,,,,,,,,,,,,,,,,,,,,,,,Y,,,,,,,


Now, let's make a new dataframe `mel` as a copy of `acnc` after filtering it by the variable `Town_City` to only select the charities and nonprofits in the city of Melbourne :

In [8]:
mel = acnc[acnc.Town_City.str.contains('melbourne', case=False)][['ABN', 'Charity_Legal_Name', 'Address_Line_1', 
                                                                  'Address_Line_2', 'Address_Line_3', 'Town_City', 
                                                                  'State', 'Postcode', 'Country', 
                                                                  'Date_Organisation_Established', 'Charity_Size']].copy()

I did two things here: I filtered the `acnc` dataframe by the `Town_City` variable, and then I only selected 11 useful columns out of the 60 we initially had. `copy()` makes sure we made a proper copy of the filtered `acnc` dataframe.

I didn't use `acnc[acnc.Town_City == 'Melbourne']` here because I suspected it may have been written in different ways. To make sure this was necessary :

In [10]:
mel.Town_City.value_counts()

Melbourne                       1779
MELBOURNE                        475
East Melbourne                   115
South Melbourne                  104
North Melbourne                   87
Port Melbourne                    48
West Melbourne                    32
SOUTH MELBOURNE                   20
EAST MELBOURNE                    18
NORTH MELBOURNE                    8
WEST MELBOURNE                     6
PORT MELBOURNE                     5
Royal Melbourne Hospital           4
Melbourne University               3
Melbourne Airport                  2
University Of Melbourne            2
University of Melbourne            2
Melbourne Officer                  1
Melbourne Aiport                   1
THE UNIVERSITY OF MELBOURNE        1
UNIVERSITY OF MELBOURNE            1
The University of Melbourne        1
Hawthorne East, Melbourne          1
Melbourne North                    1
West melbourne                     1
The Royal Melbourne Hospital       1
Melbourne Law Courts               1
s

As we can see above, the column contained different ways of indicating that a charity is indeed situated in Melbourne, some of which using suburbs or even very specific places in the city like The University of Melbourne. By using `acnc.Town_City.str.contains('melbourne', case=False)` , we insured all of the above charities are accounted for (otherwise we would only get the 1779 ones correctly labeled).

Let's see how our new `mel` dataframe looks like :

In [11]:
mel.head()   # 2722 rows and 11 columns

,ABN,Charity_Legal_Name,Address_Line_1,Address_Line_2,Address_Line_3,Town_City,State,Postcode,Country,Date_Organisation_Established,Charity_Size
15,11004285956,Australian Nurses Memorial Centre,G Se 11 431 St Kilda Rd,,,Melbourne,Victoria,3004,Australia,31/12/1949,Small
123,11128866419,Australian Community Philanthropy Ltd,L 15 1 Collins St,,,Melbourne,Victoria,3000,Australia,01/01/2007,Small
125,11129013305,Nafsika Stamoulis Hellenic Museum Limited,280 William St,,,Melbourne,Victoria,3000,Australia,01/01/2007,Large
126,11129790467,Doxa Community Club Incorporated,"Ground Floor, Suite 2, 355 Spencer Street",,,West Melbourne,VIC,3003,Australia,01/01/1992,Large
135,11137037677,Financial Basics Community Foundation Limited,L 7 525 Flinders St,,,Melbourne,Victoria,3000,Australia,01/01/2009,Small


And now, let's add a new column `Full_Address` containing the full address :

In [12]:
mel['Full_Address'] = mel['Address_Line_1'].str.cat( mel[['Address_Line_2', 'Address_Line_3', 'Town_City']], sep=' ')

`str.cat()` works here because all of these columns are of type object or string.

Here's an example of the very first full address in `mel` :

In [13]:
mel.Full_Address.iloc[0]

'G Se 11 431 St Kilda Rd   Melbourne'

One more thing: some of these full addresses only contain the Post Office Box number (mentioned as GPO Box or Po Box). These addresses are completely useless to us because they don't refer to an existing place. Here's an example :

In [14]:
mel[mel.Full_Address.str.contains('po box', case=False)].Full_Address.iloc[0]

'GPO Box 2307   Melbourne'

We need to remove these records (or rows) before proceeding :

In [15]:
mel = mel[~mel.Full_Address.str.contains('po box', case=False)].copy()

One last thing: some addresses contain the character `/` which will break any URL we want to generate from the full address. We need to substitute any slash symbol with a space :

In [16]:
mel.Full_Address = mel.Full_Address.str.replace('/', ' ')

---

# Exploring Google Maps :

Before any web scraping job, it is essential to explore the website you want to extract data from. In our case, it's [Google Maps](https://www.google.com/maps).

First, let's study how searching for a full address using the search bar inside Google Maps affects the URL of the result page. For this, I'll go with the fictitious address `Grinch house mount crumpit whoville` because I want Google Maps to return no results :

![google_maps](https://cdn-images-1.medium.com/max/800/1*CMzGOjASfF8x_DJAF1qBuQ.png)

As you can see above, we get `www.google.com/maps/search/` followed by  the address we searched for. In other words, if we want to search for an address XYZ within Google Maps, all we have to do is use the URL `www.google.com/maps/search/XYZ`, without having to interact with the search bar itself.

The idea here is to generate a new column within `mel` where we combine `www.google.com/maps/search/` with every `Full_Address` we have in our dataframe `mel`, and then have Selenium iterate over them visiting the URLs one after the other.

Let's create that new `Url` column :

In [17]:
mel['Url'] = ['https://www.google.com/maps/search/' + i for i in mel['Full_Address'] ]

In [18]:
mel[mel.Full_Address.str.contains('G Se 11 431 St Kilda Rd', case=False)]

,ABN,Charity_Legal_Name,Address_Line_1,Address_Line_2,Address_Line_3,Town_City,State,Postcode,Country,Date_Organisation_Established,Charity_Size,Full_Address,Url
15,11004285956,Australian Nurses Memorial Centre,G Se 11 431 St Kilda Rd,,,Melbourne,Victoria,3004,Australia,31/12/1949,Small,G Se 11 431 St Kilda Rd Melbourne,https://www.google.com/maps/search/G Se 11 431...


Now that we have a column containing all the URLs we want to crawl, let's take a look at the address `G Se 11 431 St Kilda Rd Melbourne` for example. The link would be (copy paste it in your browser):

`https://www.google.com/maps/search/G Se 11 431 St Kilda Rd Melbourne`

The above link gives us :

![nurses_memorial](https://cdn-images-1.medium.com/max/800/1*bS5q2yPhnzbqpN9nR3odYQ.png)

The above address corresponds to the charity **Australian Nurses Memorial Centre**. Let's search for it on Google Maps by name :

![charity](https://cdn-images-1.medium.com/max/800/1*lImFiCK4_KuMJ8huPm_EPw.png)

We get the exact same spot, but not the same coordinates in the URL. That's because the coordinates in the URL are linked to how the map is centered and not to the marker (they change if you zoom in or out). That's why we're going to extract the coordinates directly from the source code of the page itself.

To view the source code, right click on a blank space within the page (outside of the map) and choose View Page Source (CTRL+U or Command+U in Mac). Now search for **-37.8** or **144.9** within the source page :

![source_code1](https://cdn-images-1.medium.com/max/800/1*17IjuHiRMK4vVhgwmKy5MA.png)

You'll find the coordinates we are seeking in many places throughout the hot mess that is the source code. But they are mostly useful to us if they are comprised within an HTML tag we can target. Luckily, there is one meta tag we can make useful here :

![source_code2](https://cdn-images-1.medium.com/max/800/1*7UsaC4LjuxdXl41TQHtc_Q.png)

For now, let's note that it's a `meta` tag with an attribute `content` containing the URL we want to extract, and an attribute `itemprop` with the value `image` which can be used to identify and target this particular `meta` tag.

Now all we have to do is use Selenium to visit each URL in `mel.Url` and target this `meta` tag in order to extract the value of its attribute `content`.

---

# Using Selenium :

Here is the code we'll use to extract the URLs containing the coordinates from Google Maps :

In [ ]:
Url_With_Coordinates = []

option = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
option.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome("C:\\chromedriver.exe", options=option)

for url in tqdmn(mel.Url, leave=False):
    driver.get(url)
    Url_With_Coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))

driver.close()

import csv

with open('Url_With_Coordinates.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(Url_With_Coordinates)

To follow along with these explanations, make sure to click `View` > `Toggle Line Numbers` in your notebook :

* **Line 1**: we make an empty list called `Url_With_Coordinates`, which we will fill later on with (you guessed it) the URLs we want to extract ;
* **Lines 3-5** : prefs to run the Webdriver without javascript and images. This way the code will take much less time to load webpages. Obviously, this isn't a good choice if what you want to extract relies on javascript ;
* **Line 7**: make sure to specify where you put the chromedriver.exe file in your machine. In my case I put it in the C drive for simplicity. Note that backlashes `\` in the path need to be doubled `\\` for the path to be recognized ;
* **Line 9**: this for loop is iterating over the `mel.Url` series. The `tqdmn()` wrapping our iterable adds a progress bar right after the cell is executed. Its parameter `leave=False` makes sure the bar goes away once the operation is finished ;
* **Line 10**: for each URL in `mel.Url`, the webdriver opens that URL (for the first URL you'll see a chrome window open, and then you'll notice it going from URL to URL until `mel.Url` is finished) ;
* **Line 11**: first, we search for our `meta` tag using `driver.find_element_by_css_selector` and we identify the tag by `meta[itemprop=image]`. After that, we extract the value of the attribute `content` using `.get_attribute('content')`. The result of this operation (the URL containing the coordinates) is then added to the `Url_With_Coordinates` list using `append()` ;
* **Line 13**: we close the webdriver (chrome window) after the script is finished (this is a good practice) ;
* **Lines 17-19**: we write our list `Url_With_Coordinates` into a CSV file just in case.

**NB: the next time you run this notebook, you don't have to rerun the webscraping code all over again** because we have saved the result in a CSV file called `Url_With_Coordinates.csv`. Let's read in that file instead :

In [19]:
import csv

with open('Url_With_Coordinates.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for i in reader:
        Url_With_Coordinates = i
        break

Now let's see what the `Url_With_Coordinates` list looks like :

In [20]:
Url_With_Coordinates[:5]

['https://maps.google.com/maps/api/staticmap?center=-37.8386737%2C144.97706&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=uFBzLNXh4_xAkoqxe9tXB97_zFk',
 'https://maps.google.com/maps/api/staticmap?center=-37.8139436%2C144.9734554&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=ZDtMkZz6Vw066nVDdUtFWzsWuuE',
 'https://maps.google.com/maps/api/staticmap?center=-37.8125657%2C144.9570751&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=KWS2gZbSdewzujGQh9qmodbP9Sc',
 'https://maps.google.com/maps/api/staticmap?center=-37.8119391%2C144.950558&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=Ma5nOuERyqGe7J9Ji6P2WXUNnpA',
 'https://maps.google.com/maps/api/staticmap?center=-37.820694%2C144.957151&zoom=17&size=256x256&language=en&sensor=false&client=google-maps-frontend&signature=S_71-6_IXI9ePcFPGADFKats1WI']

Let's add this list as a column in our `mel` dataframe :

In [21]:
mel['Url_With_Coordinates'] = Url_With_Coordinates

But how are we supposed to extract just the coordinates from these URLs? Here's a visual explanation on how to use Python's `split()` method for this purpose :

![split_method](https://cdn-images-1.medium.com/max/800/1*e0mPQ0M3FGf6dhxBSvILXQ.png)

Which translates in code to the following :

`url.split('?center=')[1].split('&zoom=')[0].split('%2C')`

Output:
[-37.8386737, 144.97706]

Now using the code above, we're going to add two new columns to our `mel` dataframe: `lat` for latitudes and `long` for longitudes :

In [22]:
mel['lat'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in mel['Url_With_Coordinates'] ]
mel['long'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in mel['Url_With_Coordinates'] ]

IndexError: list index out of range

Most likely, the above code gave you an error `list index out of range`. What this error means is that the `split()` method didn't work as intended with some URLs in the column `Url_With_Coordinates`. Perhaps some URLs  didn't have the keywords we used for the `split()` method. Let's look for the URLs that lack `&zoom=` for example : 

In [23]:
mel[~mel.Url_With_Coordinates.str.contains('&zoom=')]

,ABN,Charity_Legal_Name,Address_Line_1,Address_Line_2,Address_Line_3,Town_City,State,Postcode,Country,Date_Organisation_Established,Charity_Size,Full_Address,Url,Url_With_Coordinates
33030,50662852831,The Trustee For Poaal Benevolent Fund,L 6 21 Victoria St,,,MELBOURNE,VIC,3000,,01/01/2009,,L 6 21 Victoria St MELBOURNE,https://www.google.com/maps/search/L 6 21 Vict...,//www.gstatic.com/images/branding/product/1x/m...
38685,57533056318,Alternative Technology Association Inc.,L 1 Se 1 39 Little Collins St,,,Melbourne,Victoria,3000,Australia,,Large,L 1 Se 1 39 Little Collins St Melbourne,https://www.google.com/maps/search/L 1 Se 1 39...,//www.gstatic.com/images/branding/product/1x/m...
44144,64162498651,AUSTRALIAN INSTITUTE OF ARCHITECTS FOUNDATION,L 1 Se 1 41 Exhibition St,,,Melbourne,Victoria,3000,Australia,21/02/2013,Small,L 1 Se 1 41 Exhibition St Melbourne,https://www.google.com/maps/search/L 1 Se 1 41...,//www.gstatic.com/images/branding/product/1x/m...
54013,76140094882,Embark Australia Ltd,L 1 Se 1 39 Little Collins St,,,Melbourne,Victoria,3000,Australia,01/01/2009,Small,L 1 Se 1 39 Little Collins St Melbourne,https://www.google.com/maps/search/L 1 Se 1 39...,//www.gstatic.com/images/branding/product/1x/m...
56810,79578875531,Philanthropy Australia Ltd,L 2 55 Collins St,,,Melbourne,Victoria,3000,Australia,01/01/1975,Large,L 2 55 Collins St Melbourne,https://www.google.com/maps/search/L 2 55 Coll...,//www.gstatic.com/images/branding/product/1x/m...


As we can see here, we have 5 instances where the extracted URL starts with `//www.gstatic.com/images ...` (hence the error we got) :

In [24]:
list(mel[~mel.Url_With_Coordinates.str.contains('&zoom=')].Url_With_Coordinates)

['//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png']

For simplicity, we'll remove these 5 from `mel` :

In [25]:
mel = mel[mel.Url_With_Coordinates.str.contains('&zoom=')].copy()

And now, let's rerun the code that adds two columns `lat` and `long` to our `mel` dataframe :

In [26]:
mel['lat'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in mel['Url_With_Coordinates'] ]
mel['long'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in mel['Url_With_Coordinates'] ]

It worked! Here's how our `mel` dataframe looks like, where each charity or nonprofit gets it latitude and longitude values :

In [27]:
mel.head()

,ABN,Charity_Legal_Name,Address_Line_1,Address_Line_2,Address_Line_3,Town_City,State,Postcode,Country,Date_Organisation_Established,Charity_Size,Full_Address,Url,Url_With_Coordinates,lat,long
15,11004285956,Australian Nurses Memorial Centre,G Se 11 431 St Kilda Rd,,,Melbourne,Victoria,3004,Australia,31/12/1949,Small,G Se 11 431 St Kilda Rd Melbourne,https://www.google.com/maps/search/G Se 11 431...,https://maps.google.com/maps/api/staticmap?cen...,-37.8386737,144.97706
123,11128866419,Australian Community Philanthropy Ltd,L 15 1 Collins St,,,Melbourne,Victoria,3000,Australia,01/01/2007,Small,L 15 1 Collins St Melbourne,https://www.google.com/maps/search/L 15 1 Coll...,https://maps.google.com/maps/api/staticmap?cen...,-37.8139436,144.9734554
125,11129013305,Nafsika Stamoulis Hellenic Museum Limited,280 William St,,,Melbourne,Victoria,3000,Australia,01/01/2007,Large,280 William St Melbourne,https://www.google.com/maps/search/280 William...,https://maps.google.com/maps/api/staticmap?cen...,-37.8125657,144.9570751
126,11129790467,Doxa Community Club Incorporated,"Ground Floor, Suite 2, 355 Spencer Street",,,West Melbourne,VIC,3003,Australia,01/01/1992,Large,"Ground Floor, Suite 2, 355 Spencer Street We...",https://www.google.com/maps/search/Ground Floo...,https://maps.google.com/maps/api/staticmap?cen...,-37.8119391,144.950558
135,11137037677,Financial Basics Community Foundation Limited,L 7 525 Flinders St,,,Melbourne,Victoria,3000,Australia,01/01/2009,Small,L 7 525 Flinders St Melbourne,https://www.google.com/maps/search/L 7 525 Fli...,https://maps.google.com/maps/api/staticmap?cen...,-37.820694,144.957151


Let's map these coordinates to see how accurate they can be.

---

# Using Folium for coordinates mapping :

Here's the code we'll use to map these coordinates into a Melbourne map with the following color coding (charity size depends on annual income) :
* **Red**: Large charities (1 Million AUD or more) ;
* **Purple**: Medium charities (between 250.000 AUD and 1 Million AUD) ;
* **Orange**: Small charities (less than 250.000 AUD);
* **Grey**: no data.

In [28]:
from IPython.display import IFrame

mel_large  = mel[mel.Charity_Size == 'Large']
mel_medium = mel[mel.Charity_Size == 'Medium']
mel_small  = mel[mel.Charity_Size == 'Small']
mel_other  = mel[mel.Charity_Size == '']

mel_map = folium.Map( [-37.8, 145], tiles='CartoDB positron' )
   
for lat, long, name, full_address in zip(mel_large.lat, mel_large.long, mel_large.Charity_Legal_Name, mel_large.Full_Address):
    folium.Marker( [lat, long], 
                   icon=folium.CustomIcon( icon_image='https://i.imgur.com/CYx04oC.png', icon_size=(10,10) ), popup=name+'\n\n'+full_address ).add_to(mel_map)

for lat, long, name, full_address in zip(mel_medium.lat, mel_medium.long, mel_medium.Charity_Legal_Name, mel_medium.Full_Address):
    folium.Marker( [lat, long], 
                   icon=folium.CustomIcon( icon_image='https://imgur.com/Rzs4Zpa.png', icon_size=(8,8) ), popup=name+'\n\n'+full_address ).add_to(mel_map)
    
for lat, long, name, full_address in zip(mel_small.lat, mel_small.long, mel_small.Charity_Legal_Name, mel_small.Full_Address):
    folium.Marker( [lat, long], 
                   icon=folium.CustomIcon( icon_image='https://imgur.com/6TWrNOY.png', icon_size=(6,6) ), popup=name+'\n\n'+full_address ).add_to(mel_map)
    
for lat, long, name, full_address in zip(mel_other.lat, mel_other.long, mel_other.Charity_Legal_Name, mel_other.Full_Address):
    folium.Marker( [lat, long], 
                   icon=folium.CustomIcon( icon_image='https://imgur.com/C1MXk3r.png', icon_size=(4,4) ), popup=name+'\n\n'+full_address ).add_to(mel_map)


mel_map.save('mel_map.html')
IFrame(src='mel_map.html', width='100%', height=500)

I'm not going into great detail on how to use Folium, but I'll just clarify some points here :

* I used `CartoDB positron` for this map because it provides very low contrast against the colored markers (it makes them more visible). Using the default tiles `OpenStreetMap` makes it difficult to see the markers ;

* I changed the size of the marker based on the size of the charity using the parameter `icon_size=(..,..)` of `folium.CustomIcon`. The reason behind this is to prevent charities that reside in the same building from masking each other. Because Large markers are drawn first, smaller markers are drawn on on top of them so that overlapping charities can still be distinguished ;

* I used custom markers (hosted on imgur) because the default one tends to slow navigation down a lot since we have around 2000 markers on the map. For custom markers, you can provide a URL to the image you want to use, or the path to the image file on your machine ;

* If you click on a marker, it gives you the name of the charity and its address, so you can verify if the positioning is correct ;

* In the code above, I could have used just `mel_map` instead of `mel_map.save('mel_map.html')` followed by `IFrame(src='mel_map.html', width='100%', height=500)`, but when the number of markers is big, it's better to save the map as an HTML file and then open it using `IFrame()` (otherwise you'll get a blank map).

---

# How accurate and reliable is this method? (bonus test)

This is a very legitimate question. Obviously, the best way to get coordinates out of street addresses is from a reputable API like Google Maps's or Baidu's, but these options can cost money. 

The accuracy of this method is heavily influenced by the accuracy and correctness of the street addresses provided. For instance, in our example above, you'll notice one marker thrown away in the middle of the Indian Ocean. Upon examination, the address `65 Macarae Road Melbourne` is actually supposed to be `65 Mcrae Road Melbourne`, hence the error.

To empirically test our method, we are going to use a dataset containing both the street addresses and the coordinates of thousands of businesses in Washington DC. We'll proceed to take a random sample of 500 business addresses and then we'll use our method to generate coordinates out of them. After that, we'll compare them to the actual coordinates listed in the dataset. The dataset we'll use for our test can be downloaded here : 

* [Basic Business License in Last 30 Days](https://opendata.dc.gov/datasets/a9ac9f8d9d1546e8bd6610b48cb29810/data): opendata.dc.gov dataset, **6418** records (as of 26/01/2020), 2.64 Mb

In [65]:
dc = pd.read_csv('data/Basic_Business_License_in_Last_30_Days.csv')       # provide your own path to your file

We take a sample of 500 records out of the dataset where `SITE_ADDRESS`, `LATITUDE` and `LONGITUDE` are all not NULL (I used the parameter `random_seet` here so you'll get the same sample as I did. Remove it to get a new random sample each time you run this cell) :

In [66]:
dc = dc[(~dc.SITE_ADDRESS.isnull()) & (~dc.LATITUDE.isnull()) & (~dc.LONGITUDE.isnull()) ].copy().sample(500, random_state = 1)

dc.head(3)

,BILLING_ADDRESS,BILLING_ADDRESS_CITY_STATE_ZIP,AGENT_ENTITY,AGENT_FIRST_NAME,AGENT_LAST_NAME,AGENT_MIDDLE_NAME,AGENT_PHONE,SITE_ADDRESS,BBL_CITY_STATE_ZIP,BILLING_NAME,CUSTOMER_NUMBER,DC_ADDR_FLAG,LICENSECATEGORY,ENTITY_NAME,FIRST_NAME,LAST_NAME,DCS_ADD_DATE,LICENSE_END_DATE,LICENSE_ISSUE_DATE,LICENSE_START_DATE,LICENSE_CATEGORY_TEXT,LICENSE_CATEGORY_CODE,LICENSESTATUS,PHONE_NUMBER,OBJECTID,CITY,STATE,ZIP,BBL_LICENSE_FACT_ID,DCS_LAST_MOD_DTTM,LATITUDE,LONGITUDE,XCOORD,YCOORD,ZIPCODE,MARADDRESSREPOSITORYID,WARD,ANC,SMD,DISTRICT,PSA,NEIGHBORHOODCLUSTER,BUSINESSIMPROVEMENTDISTRICT,MAINSTREET
5814,SUITE/APT #400,451 Hungerford Drive ROCKVILLE MD 20850,CORPORATION SERVICE COMPANY,NaN,NaN,NaN,2024083121,1100 CONNECTICUT AVE NW,WASHINGTON DC 20036,NaN,400314000777,Y,General Business Licenses,PLANET DEPOS LLC,NaN,NaN,NaN,2022-01-31T00:00:00.000Z,2020-01-14T00:00:00.000Z,2020-02-01T00:00:00.000Z,General Business,4003.0,Ready to Batch Print,NaN,184894,WASHINGTON,DC,20036,88432196,2020-01-25T05:22:13.000Z,38.904073,-77.040638,396475.11,137358.15,NaN,243303.0,2.0,2B,2B06,SECOND,207.0,Cluster 6,Golden Triangle BID,NaN
5569,NaN,"310 W. 20th Street, Kansas City MO 64108",CATHERINE READ,NaN,NaN,NaN,NaN,1015 15TH ST NW,WASHINGTON DC 20005,NaN,400218000536,Y,Charitable Solicitation,Center for Victims of Torture,NaN,NaN,NaN,2020-03-31T00:00:00.000Z,2020-01-04T00:00:00.000Z,2018-04-01T00:00:00.000Z,General Business,4002.0,Ready to Renew,NaN,184039,WASHINGTON,DC,20005,88433876,2020-01-25T05:22:13.000Z,38.902997,-77.034250,397029.16,137238.53,NaN,240258.0,2.0,2F,2F05,SECOND,207.0,Cluster 8,Downtown BID,NaN
379,NaN,12 Pondview Road Chester NJ 07930,Patrick J. Tangney - Logan Title,Patrick,Tangney,J,2024002508,920 I ST NW,Washington DC 20001,NaN,500518802633,Y,One Family Rental,Gregory Savettiere,NaN,NaN,NaN,2022-02-28T00:00:00.000Z,2020-01-03T00:00:00.000Z,2020-03-01T00:00:00.000Z,Housing: Residential,5005.0,Ready to Batch Print,NaN,23957,Washington,DC,20001,88468710,2020-01-25T05:22:15.000Z,38.900467,-77.024568,397868.89,136957.35,NaN,307522.0,2.0,2C,2C01,SECOND,209.0,Cluster 8,Downtown BID,NaN


We generate our Google Maps search URL :

In [67]:
dc['URL'] = ['https://www.google.com/maps/search/' + i +' Washington DC' for i in dc['SITE_ADDRESS'] ]

We run our Senelium web scraper :

In [68]:
DC_Url_With_Coordinates = []

option = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
option.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome("C:\\chromedriver.exe", options=option)

for url in tqdmn(dc.URL, leave=False):
    driver.get(url)
    DC_Url_With_Coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))

driver.close()

We add the URLs to the `dc` dataset :

In [69]:
dc['DC_Url_With_Coordinates'] = DC_Url_With_Coordinates

And then the latitudes and longitudes (GEN stands for Generated Latitudes and Longitudes) :

In [70]:
dc['GEN_LATITUDE'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in dc['DC_Url_With_Coordinates'] ]
dc['GEN_LONGITUDE'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in dc['DC_Url_With_Coordinates'] ]

Finally, let's map them both: actual coordinates using pink and generated coordinates using orange :

In [71]:
from IPython.display import IFrame

dc_map = folium.Map( [38.89, -77.15], tiles='CartoDB positron' )
   
for lat, long, full_address in zip(dc.LATITUDE, dc.LONGITUDE, dc.SITE_ADDRESS):
    folium.Marker( [lat, long], 
                   icon=folium.CustomIcon( icon_image='https://imgur.com/joZ4jCn.png', icon_size=(12,12) ), popup=full_address ).add_to(dc_map)

for lat, long, full_address in zip(dc.GEN_LATITUDE, dc.GEN_LONGITUDE, dc.SITE_ADDRESS):
    folium.Marker( [lat, long], 
                   icon=folium.CustomIcon( icon_image='https://imgur.com/6TWrNOY.png', icon_size=(9,9) ), popup=full_address ).add_to(dc_map)
    
dc_map.save('dc_map.html')
IFrame(src='dc_map.html', width='100%', height=500)

As we can see above, the generated coordinates are very close to the actual coordinates in this random sample of 500 businesses (out of 6418).